Our goal is to create a regression model, which predicts the price of a given property based on a variety of different predictors/features. We first import the data and we perform some feature enginnering, in order to deal with the null values of some columns. Then we do some exploratory data analysis on the dataset, so that we can understand our data and features in depth. 

Note that we have two datasets: one dataset (called 'data') which contains labeled data, that is data with given response variable (Sale price), and one dataset ('unlabeled') which contains unlabeled data. We use the first dataset to train and test our model and the second dataset is used from our model to make predictions on -previously- unseen data.

Obviously, the last column 'SalePrice' is the numerical column corresponding to the response variable. The rest columns correspond to features (or predictors or covariates).

# Feature enginnering on the training data

In [64]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn 

In [65]:
data = pd.read_csv('/Users/dimitriosmichailidis/Downloads/train.csv',index_col='Id')
test_data = pd.read_csv('/Users/dimitriosmichailidis/Downloads/test.csv',index_col='Id')

In [66]:
features = data.drop('SalePrice',axis = 1)
labels = data['SalePrice']

In [67]:
# We split the data into two different dataframes, one containing the numeric columns and one containing the
# categorical columns 

data_num = features._get_numeric_data()
data_cat = features.drop(list(data_num.columns), axis = 1)

#### Deal with missing values in the categorical columns of our data

In [68]:
# In the following series we see that there are a few columns with missing values

data_cat.isnull().sum()

MSZoning            0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinType2       38
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu       690
GarageType         81
GarageFinish       81
GarageQual         81
GarageCond         81
PavedDrive          0
PoolQC           1453
Fence            1179
MiscFeature      1406
SaleType            0
SaleCondition       0
dtype: int64

In [69]:
# We define a function which replaces the null categorical values of a column, with the string 'No_'+column_name.
# This is used when the meaning of the null values of the column is that the house corresponding to the row does
# not possess the facility described in the column. We do not simply fill the null values with the string 'No'
# because this will create confusion when we want to convert the categorical columns to numerical by using dymmy
# variables

def fill_na_not_exist(data_frame,column_name):
    
    data_frame[column_name].fillna('No_'+column_name,inplace = True)
    

# We create another function for substituting missing categorical values, with the most common value of the column.

def fill_na_most_common(data_frame,column_name):
    
    most_common_value = data_frame[column_name].mode()[0]
    
    #data_frame[column_name + '_Imputed'] = data[column_name]
    data_frame[column_name].fillna(most_common_value,inplace = True)

In [70]:
# According to the description of the data set, the null values of the 'Alley' column indicate that there is no
# alley access to the house. Hence shall replace the null values with a string indicating there is no alley, by
# using the function above.

fill_na_not_exist(data_cat,'Alley')
fill_na_not_exist(data_cat,'MiscFeature')

In [71]:
# Fill the null values of a bunch of column referring to the Basement, the Fireplaces, the Garage, the Pool 
# the Fence and miscellaneous features of the house, the Electrical type and the material of the Masonry venner

for column in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',
              'GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence',
               'Electrical','MasVnrType']:
    
    fill_na_not_exist(data_cat,column)

In [72]:
# There are no more missing values indeed

data_cat.isnull().sum()

MSZoning         0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
KitchenQual      0
Functional       0
FireplaceQu      0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PavedDrive       0
PoolQC           0
Fence            0
MiscFeature      0
SaleType         0
SaleCondition    0
dtype: int64

In [73]:
# For the sake of consistency we rename the data_cat dataframe with no null values and we denote it by
# no_nan_data_cat. This data frame has 1460 rows and 43 columns.

no_nan_data_cat = data_cat

#### We shall deal with the categorical data. We will either create dummy variables or we will use OneHotEncoder. Then we shall merge the dataframe no_nan_data_num with the dataframe consisting of the dummy variables arising from mno_nan_data_cat

In [74]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [75]:
imputer = SimpleImputer(strategy='mean')

In [76]:
imputer.fit(data_num)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [77]:
imputer.statistics_

array([5.68972603e+01, 7.00499584e+01, 1.05168281e+04, 6.09931507e+00,
       5.57534247e+00, 1.97126781e+03, 1.98486575e+03, 1.03685262e+02,
       4.43639726e+02, 4.65493151e+01, 5.67240411e+02, 1.05742945e+03,
       1.16262671e+03, 3.46992466e+02, 5.84452055e+00, 1.51546370e+03,
       4.25342466e-01, 5.75342466e-02, 1.56506849e+00, 3.82876712e-01,
       2.86643836e+00, 1.04657534e+00, 6.51780822e+00, 6.13013699e-01,
       1.97850616e+03, 1.76712329e+00, 4.72980137e+02, 9.42445205e+01,
       4.66602740e+01, 2.19541096e+01, 3.40958904e+00, 1.50609589e+01,
       2.75890411e+00, 4.34890411e+01, 6.32191781e+00, 2.00781575e+03])

In [78]:
# DataFrame containing numeric values with the null values filled by the imputer

data_num_tr = pd.DataFrame(imputer.transform(data_num),columns=data_num.columns,
                          index = data_num.index)

In [79]:
merged_data = pd.merge(data_num_tr,no_nan_data_cat,on='Id')

In [80]:
# We create a pipeline that performs a series of transformations on our numeric data.
# First it applies the SimpleImputer class in order to fill the null values with the mean value of the
# corresponding column.
# Then it performs feature scaling by using the MinMaxScaler.

num_pipeline = Pipeline([('scl',MinMaxScaler())
                        ])

In [81]:
num_attributes = list(data_num)
cat_attributes = list(data_cat)

In [82]:
# We create a ColumnTransformer instance in order to deal with both the numeric and categorical attributes

full_pipeline = ColumnTransformer([
    ('num',num_pipeline,num_attributes),
    ('cat',OneHotEncoder(),cat_attributes)
])

In [83]:
data_prepared = full_pipeline.fit_transform(merged_data)

# Model selection and Cross-Validation

In order to find the best model possible, we shall use both the train_test_split method and the K-fold cross-validation method

In [84]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [85]:
X_train, X_test, y_train, y_test = train_test_split(data_prepared,labels,test_size = 0.3)

In [86]:
# We transform the sparse matrices to NumPy arrays

X_train = X_train.toarray()
X_test = X_test.toarray()

In [87]:
# We build our neural network 

model = Sequential()

model.add(Dense(255,activation = 'relu'))
model.add(Dense(255,activation = 'relu'))
model.add(Dense(255,activation = 'relu'))
model.add(Dense(255,activation = 'relu'))
model.add(Dense(255,activation = 'relu'))
model.add(Dense(255,activation = 'relu'))
model.add(Dense(255,activation = 'relu'))
model.add(Dense(255,activation = 'relu'))
model.add(Dense(255,activation = 'relu'))


model.add(Dense(1))

model.compile(optimizer = 'rmsprop',loss = 'mse')

In [88]:
model.fit(X_train,y_train,epochs=500)

Epoch 1/500
32/32 [==============================] - 0s 4ms/step - loss: 13209904128.0000
Epoch 2/500
32/32 [==============================] - 0s 5ms/step - loss: 3693873152.0000
Epoch 3/500
32/32 [==============================] - 0s 5ms/step - loss: 2962527232.0000
Epoch 4/500
32/32 [==============================] - 0s 6ms/step - loss: 2403659776.0000
Epoch 5/500
32/32 [==============================] - 0s 4ms/step - loss: 2377308160.0000
Epoch 6/500
32/32 [==============================] - 0s 4ms/step - loss: 2238169344.0000
Epoch 7/500
32/32 [==============================] - 0s 4ms/step - loss: 2405693952.0000
Epoch 8/500
32/32 [==============================] - 0s 4ms/step - loss: 2076085504.0000
Epoch 9/500
32/32 [==============================] - 0s 4ms/step - loss: 1814205440.0000
Epoch 10/500
32/32 [==============================] - 0s 4ms/step - loss: 2005121408.0000
Epoch 11/500
32/32 [==============================] - 0s 4ms/step - loss: 2214732032.0000
Epoch 12/500
32/32

32/32 [==============================] - 0s 5ms/step - loss: 677380352.0000
Epoch 93/500
32/32 [==============================] - 0s 6ms/step - loss: 504140160.0000
Epoch 94/500
32/32 [==============================] - 0s 4ms/step - loss: 1030173952.0000
Epoch 95/500
32/32 [==============================] - 0s 3ms/step - loss: 418919296.0000
Epoch 96/500
32/32 [==============================] - 0s 4ms/step - loss: 625455552.0000
Epoch 97/500
32/32 [==============================] - 0s 6ms/step - loss: 579908928.0000
Epoch 98/500
32/32 [==============================] - 0s 5ms/step - loss: 799076928.0000
Epoch 99/500
32/32 [==============================] - 0s 3ms/step - loss: 711756672.0000
Epoch 100/500
32/32 [==============================] - 0s 3ms/step - loss: 613984768.0000
Epoch 101/500
32/32 [==============================] - 0s 3ms/step - loss: 684549056.0000
Epoch 102/500
32/32 [==============================] - 0s 3ms/step - loss: 674438400.0000
Epoch 103/500
32/32 [=========

32/32 [==============================] - 0s 3ms/step - loss: 387589696.0000
Epoch 184/500
32/32 [==============================] - 0s 3ms/step - loss: 412226048.0000
Epoch 185/500
32/32 [==============================] - 0s 3ms/step - loss: 376313056.0000
Epoch 186/500
32/32 [==============================] - 0s 3ms/step - loss: 369017248.0000
Epoch 187/500
32/32 [==============================] - 0s 3ms/step - loss: 457454400.0000
Epoch 188/500
32/32 [==============================] - 0s 3ms/step - loss: 330191776.0000
Epoch 189/500
32/32 [==============================] - 0s 3ms/step - loss: 358710336.0000
Epoch 190/500
32/32 [==============================] - 0s 3ms/step - loss: 417749920.0000
Epoch 191/500
32/32 [==============================] - 0s 3ms/step - loss: 370036864.0000
Epoch 192/500
32/32 [==============================] - 0s 3ms/step - loss: 442764704.0000
Epoch 193/500
32/32 [==============================] - 0s 3ms/step - loss: 342274240.0000
Epoch 194/500
32/32 [===

32/32 [==============================] - 0s 3ms/step - loss: 223243664.0000
Epoch 275/500
32/32 [==============================] - 0s 3ms/step - loss: 345584352.0000
Epoch 276/500
32/32 [==============================] - 0s 3ms/step - loss: 294895360.0000
Epoch 277/500
32/32 [==============================] - 0s 3ms/step - loss: 251645056.0000
Epoch 278/500
32/32 [==============================] - 0s 3ms/step - loss: 286502528.0000
Epoch 279/500
32/32 [==============================] - 0s 3ms/step - loss: 323413056.0000
Epoch 280/500
32/32 [==============================] - 0s 3ms/step - loss: 266905568.0000
Epoch 281/500
32/32 [==============================] - 0s 3ms/step - loss: 227491216.0000
Epoch 282/500
32/32 [==============================] - 0s 3ms/step - loss: 335659840.0000
Epoch 283/500
32/32 [==============================] - 0s 3ms/step - loss: 274041376.0000
Epoch 284/500
32/32 [==============================] - 0s 3ms/step - loss: 316861664.0000
Epoch 285/500
32/32 [===

32/32 [==============================] - 0s 3ms/step - loss: 236547792.0000
Epoch 366/500
32/32 [==============================] - 0s 3ms/step - loss: 190290960.0000
Epoch 367/500
32/32 [==============================] - 0s 3ms/step - loss: 244218320.0000
Epoch 368/500
32/32 [==============================] - 0s 3ms/step - loss: 229492032.0000
Epoch 369/500
32/32 [==============================] - 0s 3ms/step - loss: 227303760.0000
Epoch 370/500
32/32 [==============================] - 0s 3ms/step - loss: 269479744.0000
Epoch 371/500
32/32 [==============================] - 0s 3ms/step - loss: 183295472.0000
Epoch 372/500
32/32 [==============================] - 0s 4ms/step - loss: 221508416.0000
Epoch 373/500
32/32 [==============================] - 0s 4ms/step - loss: 258237312.0000
Epoch 374/500
32/32 [==============================] - 0s 4ms/step - loss: 211651536.0000
Epoch 375/500
32/32 [==============================] - 0s 3ms/step - loss: 210493808.0000
Epoch 376/500
32/32 [===

32/32 [==============================] - 0s 3ms/step - loss: 135662160.0000
Epoch 457/500
32/32 [==============================] - 0s 4ms/step - loss: 206289984.0000
Epoch 458/500
32/32 [==============================] - 0s 6ms/step - loss: 202646256.0000
Epoch 459/500
32/32 [==============================] - 0s 4ms/step - loss: 141552128.0000
Epoch 460/500
32/32 [==============================] - 0s 3ms/step - loss: 180508576.0000
Epoch 461/500
32/32 [==============================] - 0s 3ms/step - loss: 177049984.0000
Epoch 462/500
32/32 [==============================] - 0s 5ms/step - loss: 197866272.0000
Epoch 463/500
32/32 [==============================] - 0s 3ms/step - loss: 180667568.0000
Epoch 464/500
32/32 [==============================] - 0s 3ms/step - loss: 227722800.0000
Epoch 465/500
32/32 [==============================] - 0s 3ms/step - loss: 189963984.0000
Epoch 466/500
32/32 [==============================] - 0s 3ms/step - loss: 131780456.0000
Epoch 467/500
32/32 [===

In [89]:
from sklearn.metrics import r2_score, mean_squared_error

In [90]:
predictions = model.predict(X_test)

In [91]:
print(f'The R2 of the neural network model is {r2_score(predictions,y_test).round(4)}')
print(f'The root mean squared error of the neural network model is {(mean_squared_error(predictions,y_test)**0.5).round(4)}')

The R2 of the neural network model is 0.8056
The root mean squared error of the neural network model is 31134.0244


In [92]:
y_test_df = pd.DataFrame(y_test)

In [93]:
predictions_df = pd.DataFrame(predictions,columns=['Predictions'],index=y_test_df.index)

In [94]:
predictions_and_real = pd.merge(predictions_df,y_test_df,on='Id')

In [95]:
predictions_and_real

,Predictions,SalePrice
Id,,
1419,125646.289062,124000
395,108524.664062,109000
1261,210050.609375,181000
394,87561.429688,100000
183,125773.765625,120000
...,...,...
30,88028.476562,68500
450,129135.054688,120000
1078,139040.109375,138800


It's a good idea to try different regression models, in order to see which works better. We shall try a Linear Regression model, a Tree model and a Support Vector Machine.

In [96]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [97]:
lreg = LinearRegression()
dtr = DecisionTreeRegressor()
svr = SVR()

In [98]:
lreg_model = lreg.fit(X_train,y_train)
dtr_model = dtr.fit(X_train,y_train)
svr_model = svr.fit(X_train,y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [99]:
lreg_predictions = lreg_model.predict(X_test)
dtr_predictions = dtr_model.predict(X_test)
svr_predictions = svr_model.predict(X_test)

In [100]:
# We compare the mean squared error of the new models and we will compare it to the mean squared error of
# the neural network.

print(f'The mean squared error of the Linear regression model is {mean_squared_error(lreg_predictions,y_test)**0.5}')
print(f'The mean squared error of the decision tree model is {(mean_squared_error(dtr_predictions,y_test)**0.5).round(4)}')
print(f'The mean squared error of the support vector machine model is {(mean_squared_error(svr_predictions,y_test)**0.5).round(4)}')

The mean squared error of the Linear regression model is 830651849981605.4
The mean squared error of the decision tree model is 43473.7383
The mean squared error of the support vector machine model is 79252.3452


In [101]:
r2_score(dtr_predictions,y_test)

0.6579077218577396

We notice that Decision Tree regressor performs well and its mean squared error is very close and a little less than the mean squared error of the Neural Network. The R2 of the Decicision Tree regressor is lower than the R2 score of the Neural Network which means that the Neural network explains higher percentage of the variance of our data. 

We also see that the mean squared error associated to the Linear regression model is huge and it's clear that there is no linear connection between the features and the labels.

The SVM performs much better than the linear model but still far worse than the Neural Network or the Decision Tree model.

In [102]:
# Another alternative for assesing our model is to perform K-fold cross validation. This process splits the data
# into K subsets and then it trains and evaluates the Decision tree model K times.

from sklearn.model_selection import cross_val_score

In [103]:
def display_scores(scores):
    
    print('Scores:',scores.round(6))
    print('Mean:',scores.mean().round(6))
    print('Standard deviation:',scores.std().round(6))

In [104]:
tree_scores = cross_val_score(dtr,data_prepared,labels,
                         scoring = 'neg_mean_squared_error',cv=10)

In [105]:
tree_rmse_scores = np.sqrt(-tree_scores)

In [106]:
display_scores(tree_rmse_scores)

Scores: [36761.105112 43101.473213 33987.746196 41548.612098 38285.303543
 28463.063246 29691.977353 34194.391459 56220.428761 37316.548985]
Mean: 37957.064997
Standard deviation: 7504.824253


In [107]:
# Now let's try out a Random Forest model. This is an instance of Ensemble learning as the Random Forest model works
# by training many Decision trees on random subsets of the features.

In [108]:
from sklearn.ensemble import RandomForestRegressor

In [109]:
#The hyperparameters are tuned by the Grid Search we performed below

forest_reg = RandomForestRegressor(n_estimators=100,max_features=24)

In [110]:
forest_reg_model = forest_reg.fit(data_prepared,labels)

In [111]:
forest_scores = cross_val_score(forest_reg,data_prepared,labels,
                               scoring = 'neg_mean_squared_error',cv=10)

In [112]:
forest_rmse_scores = np.sqrt(-forest_scores)

In [113]:
display_scores(forest_rmse_scores)

Scores: [20747.970177 27790.642741 24081.094552 39379.058909 37787.094947
 27321.020275 23618.65143  24330.538349 41399.269802 24183.359087]
Mean: 29063.870027
Standard deviation: 7135.649644


The Random Forest Regressor performs better than any of the previous models! We shall perform Grid Search and see of we can tune the hyperparameters of the Random Forest model in order to get better results, that is lower mean squared error.

In [114]:
from sklearn.model_selection import GridSearchCV

In [115]:
param_grid = [
    {'n_estimators':[3,10,30,50,100],'max_features':[2,4,6,8,10,16,24]},
    {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]}
]

In [116]:
grid_search = GridSearchCV(forest_reg,param_grid,cv = 5,
                          scoring='neg_mean_squared_error',
                          return_train_score=True)

In [117]:
grid_search.fit(data_prepared,labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None, max_features=24,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [2, 4, 6, 8, 10, 16, 24],
                          'n_estimators'

In [118]:
grid_search.best_params_

{'max_features': 24, 'n_estimators': 100}

# We shall now do some feature engineering on the test data and we shall make our final predictions

In [119]:
# We have already imported out test data. The difference from the training data is that 

test_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml


In [120]:
test_data_num = test_data._get_numeric_data()
test_data_cat = test_data.drop(list(test_data_num),axis = 1)

In [121]:
test_data_num

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,730.0,140,0,0,0,120,0,0,6,2010
1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,312.0,393,36,0,0,0,0,12500,6,2010
1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,482.0,212,34,0,0,0,0,0,3,2010
1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,470.0,360,36,0,0,0,0,0,6,2010
1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,506.0,0,82,0,0,144,0,0,1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,6,2006
2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,...,286.0,0,24,0,0,0,0,0,4,2006
2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,...,576.0,474,0,0,0,0,0,0,9,2006


In [122]:
test_data_cat

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,RH,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1462,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,NaN,Gar2,WD,Normal
1463,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,Fin,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1464,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,Fin,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1465,RL,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,WD,Normal
2916,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,...,CarPort,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
2917,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml


In [123]:
imputer.fit_transform(test_data_num)

array([[2.0000e+01, 8.0000e+01, 1.1622e+04, ..., 0.0000e+00, 6.0000e+00,
        2.0100e+03],
       [2.0000e+01, 8.1000e+01, 1.4267e+04, ..., 1.2500e+04, 6.0000e+00,
        2.0100e+03],
       [6.0000e+01, 7.4000e+01, 1.3830e+04, ..., 0.0000e+00, 3.0000e+00,
        2.0100e+03],
       ...,
       [2.0000e+01, 1.6000e+02, 2.0000e+04, ..., 0.0000e+00, 9.0000e+00,
        2.0060e+03],
       [8.5000e+01, 6.2000e+01, 1.0441e+04, ..., 7.0000e+02, 7.0000e+00,
        2.0060e+03],
       [6.0000e+01, 7.4000e+01, 9.6270e+03, ..., 0.0000e+00, 1.1000e+01,
        2.0060e+03]])

In [124]:
# We store the numeric data, with no null values in a new data frame

test_data_num_tr = pd.DataFrame(imputer.fit_transform(test_data_num),columns=test_data_num.columns,
                               index=test_data_num.index)

In [125]:
test_data_cat.isnull().sum()

MSZoning            4
Street              0
Alley            1352
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         16
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           44
BsmtCond           45
BsmtExposure       44
BsmtFinType1       42
BsmtFinType2       42
Heating             0
HeatingQC           0
CentralAir          0
Electrical          0
KitchenQual         1
Functional          2
FireplaceQu       730
GarageType         76
GarageFinish       78
GarageQual         78
GarageCond         78
PavedDrive          0
PoolQC           1456
Fence            1169
MiscFeature      1408
SaleType            1
SaleCondition       0
dtype: int64

In [126]:
for column in ['Alley','MiscFeature']:
    
    fill_na_not_exist(test_data_cat,column)

In [130]:
for column in ['MSZoning','Utilities','Exterior1st','Exterior2nd','MasVnrType','BsmtQual','BsmtCond',
              'BsmtExposure','BsmtFinType1','BsmtFinType2','KitchenQual','Functional','FireplaceQu',
              'GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','SaleType']:
    
    fill_na_most_common(test_data_cat,column)

In [131]:
test_data_cat.isnull().sum()

MSZoning         0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
KitchenQual      0
Functional       0
FireplaceQu      0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PavedDrive       0
PoolQC           0
Fence            0
MiscFeature      0
SaleType         0
SaleCondition    0
dtype: int64

In [132]:
# This is our final test data after all the data engineering.

merged_test_data = pd.merge(test_data_num_tr,test_data_cat,on='Id')

In [134]:
# We call the transform method on our full_pipeline 

test_data_prepared = full_pipeline.transform(merged_test_data)

# Now it's the time to make our final predictions on the test data!

In [135]:
final_predictions = forest_reg_model.predict(test_data_prepared)

In [139]:
predictions_df = pd.DataFrame(final_predictions,columns = ['SalePrice'], index=test_data_cat.index)

In [140]:
predictions_df

,SalePrice
Id,
1461,129306.16
1462,156184.00
1463,184234.40
1464,189272.00
1465,193796.64
...,...
2915,95544.83
2916,104508.50
2917,159232.60


In [145]:
predictions = predictions_df.to_csv('predictions.csv',header=True)